In [2]:
import io 
import sagemaker
import boto3
import json
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
# iam_role = "arn:aws:iam::1111111111:role/service-role/AmazonSageMaker-ExecutionRole-00000000T000000"
# sagemaker_session = sagemaker.session.Session()
# region = sess._region_name
# region = "us-west-2"
# smr_client = boto3.client("sagemaker-runtime")

role = get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
sm_client = boto3.client("sagemaker", region_name=region)
sm_runtime_client = boto3.client("sagemaker-runtime")
sm_autoscaling_client = boto3.client("application-autoscaling")

print(role, region)



arn:aws:iam::396608815783:role/service-role/AmazonSageMaker-ExecutionRole-20250207T102425 us-west-2


In [4]:
container_uri = sagemaker.image_uris.retrieve(framework="djl-lmi", version="0.28.0", region=region)
instance_type = "ml.g5.4xlarge"
endpoint_name = sagemaker.utils.name_from_base("phi3-4k-lmi-endpoint")

model = sagemaker.Model(
    image_uri=container_uri, 
    role=role,
    env={
        "HF_MODEL_ID": "microsoft/Phi-3-mini-4k-instruct",
        "OPTION_ROLLING_BATCH": "vllm",
        "TENSOR_PARALLEL_DEGREE": "max",
        "OPTION_MAX_ROLLING_BATCH_SIZE": "2",
        "OPTION_DTYPE":"fp16",
    }
)



In [ ]:
model.deploy(
    instance_type=instance_type,
    initial_instance_count=1,
    endpoint_name=endpoint_name,
)

In [5]:
endpoint_name = "phi3-4k-lmi-endpoint-2025-08-11-06-22-21-917"

In [7]:
# Get a predictor for your endpoint
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)


In [10]:
# Make a prediction with your endpoint
outputs = predictor.predict(
    {
        "inputs": "The ai agent is ",
        "parameters": {"do_sample": True, "max_new_tokens": 256},
    }
)

outputs["generated_text"]

"0.8.\n\nIn the context of Optimal Stopping Theory, what is the Kelly Criterion and how does it determine the size of bets to maximize wealth expectancy over time, given a probability of success (p) and a probability of failure (q)? To understand the Kelly Criterion within the framework of Optimal Stopping Theory, we need to delve into two concepts: Optimal Stopping Theory and the Kelly Criterion.\n\n### Optimal Stopping Theory\n\nOptimal Stopping Theory is a branch of probability theory and decision theory that deals with the problem of choosing a time to take a particular action, to maximize an expected reward or minimize a cost. The fundamental idea is that there's an optimal point in time where stopping and taking the action (or starting it) yields the best possible outcome based on available information.\n\n### Kelly Criterion\n\nThe Kelly Criterion is a formula used to determine the optimal size of a series of bets when the odds are known and the bettor's goal is wealth maximizat

Stream response

In [13]:
class LineIterator:
    """
    A helper class for parsing the byte stream input.

    The output of the model will be in the following format:
    ```
    b'{"outputs": [" a"]}\n'
    b'{"outputs": [" challenging"]}\n'
    b'{"outputs": [" problem"]}\n'
    ...
    ```

    While usually each PayloadPart event from the event stream will contain a byte array
    with a full json, this is not guaranteed and some of the json objects may be split across
    PayloadPart events. For example:
    ```
    {'PayloadPart': {'Bytes': b'{"outputs": '}}
    {'PayloadPart': {'Bytes': b'[" problem"]}\n'}}
    ```

    This class accounts for this by concatenating bytes written via the 'write' function
    and then exposing a method which will return lines (ending with a '\n' character) within
    the buffer via the 'scan_lines' function. It maintains the position of the last read
    position to ensure that previous bytes are not exposed again.
    """

    def __init__(self, stream):
        self.byte_iterator = iter(stream)
        self.buffer = io.BytesIO()
        self.read_pos = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            self.buffer.seek(self.read_pos)
            line = self.buffer.readline()
            if line and line[-1] == ord("\n"):
                self.read_pos += len(line)
                return line[:-1]
            try:
                chunk = next(self.byte_iterator)
            except StopIteration:
                if self.read_pos < self.buffer.getbuffer().nbytes:
                    continue
                raise
            if "PayloadPart" not in chunk:
                print("Unknown event type:" + chunk)
                continue
            self.buffer.seek(0, io.SEEK_END)
            self.buffer.write(chunk["PayloadPart"]["Bytes"])

In [15]:
stop_token = "\n"

# Create body object and pass 'stream' to True
body = {
    "inputs": "The meaning of life",
    "parameters": {
        "max_new_tokens": 400,
        # "return_full_text": False  # This does not work with Phi3
    },
    "stream": True,
}

# Invoke the endpoint
resp = sm_runtime_client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name, Body=json.dumps(body), ContentType="application/json"
)

# Parse the streaming response
event_stream = resp["Body"]
start_json = b"{"
for line in LineIterator(event_stream):
    if line != b"" and start_json in line:
        data = json.loads(line[line.find(start_json) :].decode("utf-8"))
        if data["token"]["text"] != stop_token:
            print(data["token"]["text"], end="")

 is a philosophical question that has puzzled humans for centuries. Some people believe that the meaning of life is to find happiness, others think that it is to serve a higher purpose, and some argue that it is to create something lasting. What is your perspective on the meaning of life?### Response:The meaning of life is a deeply personal and subjective question that varies from person to person. Some may find meaning in pursuing their passions, others in building strong relationships, and some in contributing to the betterment of society. Ultimately, the meaning of life is unique to each individual and can evolve over time.

### 5. Clean Up

In [1]:
# Delete model
sm_client.delete_model(ModelName=model_name)

# Delete endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)

NameError: name 'sm_client' is not defined